# Where's Waldo - Patch Direction Game
## Unified Streamlit + PyTorch Notebook
This script creates an interactive web application using Streamlit for a "Patch Direction Game".

It loads image pairs (a central patch and a neighboring patch) and displays them to the user. The user then guesses the direction of the neighboring patch relative to the central one using on-screen buttons. The script also loads a pre-trained Convolutional Neural Network (CNN) built with PyTorch which predicts the same direction based on the image patches. Finally, the application provides feedback on the user's guess, shows the AI model's prediction for comparison, and keeps score.

Realized by Nikola Trouhtchev, Hugo M. V. Arsenio, Dan Anghel

Requires: `torch`, `numpy`, `opencv-python` (for cv2), `torchvision`, and `streamlit`.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import os
from dataclasses import dataclass
from typing import Tuple, List
from torchvision import transforms
import streamlit as st


In [ ]:
class PatchPositionNet(nn.Module):
    def __init__(self):
        super(PatchPositionNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(6, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((4, 4)),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 8)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x


In [ ]:
@dataclass
class PatchConfig:
    patch_size: int = 32
    gap: int = 16
    jitter: int = 4
    max_image_size: Tuple[int, int] = (128, 128)
    color_drop: bool = True

transform = transforms.ToTensor()

def load_images_from_folder(folder: str, config: PatchConfig) -> List[Tuple[str, np.ndarray]]:
    images = []
    for filename in sorted(os.listdir(folder)):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, config.max_image_size)
            images.append((filename, img))
    return images

def apply_color_dropping(patch: np.ndarray) -> np.ndarray:
    keep_channel = np.random.randint(3)
    noise = np.random.normal(0, 1, patch.shape).astype(np.uint8)
    dropped = np.copy(patch)
    for c in range(3):
        if c != keep_channel:
            dropped[:, :, c] = noise[:, :, c]
    return dropped

def extract_random_patch_pair(img: np.ndarray, config: PatchConfig, apply_color_drop: bool = True) -> Tuple[np.ndarray, np.ndarray, int]:
    h, w, _ = img.shape
    margin = config.patch_size + config.gap + config.jitter
    DIRECTION_MAP = {
        0: ( 0,  1), 1: (-1,  1), 2: (-1,  0), 3: (-1, -1),
        4: ( 0, -1), 5: ( 1, -1), 6: ( 1,  0), 7: ( 1,  1)
    }
    shift = config.patch_size + config.gap
    for _ in range(20):
        if h <= 2 * margin or w <= 2 * margin:
            raise ValueError("Image is too small to extract patches with the given config.")
        x1 = np.random.randint(margin, w - margin)
        y1 = np.random.randint(margin, h - margin)
        dx_jit, dy_jit = np.random.randint(-config.jitter, config.jitter + 1, size=2)
        x1 += dx_jit
        y1 += dy_jit
        valid_directions = []
        for label, (dy, dx) in DIRECTION_MAP.items():
            x2 = x1 + dx * shift
            y2 = y1 + dy * shift
            if (0 <= x2 < w - config.patch_size and 0 <= y2 < h - config.patch_size):
                valid_directions.append((label, x2, y2))
        if not valid_directions:
            continue
        direction, x2, y2 = valid_directions[np.random.randint(len(valid_directions))]
        patch1 = img[y1:y1 + config.patch_size, x1:x1 + config.patch_size]
        patch2 = img[y2:y2 + config.patch_size, x2:x2 + config.patch_size]
        if patch1.shape != (config.patch_size, config.patch_size, 3) or patch2.shape != (config.patch_size, config.patch_size, 3):
            continue
        if apply_color_drop and config.color_drop:
            patch1 = apply_color_dropping(patch1)
            patch2 = apply_color_dropping(patch2)
        return patch1, patch2, direction
    raise ValueError("Failed to extract a valid patch pair after 20 attempts.")

def preprocess_patch_pair(p1: np.ndarray, p2: np.ndarray) -> torch.Tensor:
    t1 = transform(p1)
    t2 = transform(p2)
    return torch.cat([t1, t2], dim=0).unsqueeze(0)


In [ ]:
def seed_everything(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)


In [ ]:
config = PatchConfig(patch_size=28, gap=8, jitter=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PatchPositionNet().to(device)
model.load_state_dict(torch.load("model.pth", map_location=device))
model.eval()

images = load_images_from_folder("images", config)
direction_names = ["Right", "Top-Right", "Top", "Top-Left", "Left", "Bottom-Left", "Bottom", "Bottom-Right"]

st.title("Where's Waldo - Patch Direction Game")
idx = np.random.randint(len(images))
filename, img = images[idx]
st.write(f"Image: {filename}")

try:
    p1, p2, label = extract_random_patch_pair(img, config, apply_color_drop=False)
except ValueError:
    st.error("Image is too small for patch extraction.")
    st.stop()

bordered_p1 = cv2.copyMakeBorder(p1, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=(255, 0, 0))
bordered_p2 = cv2.copyMakeBorder(p2, 2, 2, 2, 2, cv2.BORDER_CONSTANT, value=(0, 0, 255))
combined = np.hstack([bordered_p1, bordered_p2])
st.image(combined, channels="BGR", width=256)

input_tensor = preprocess_patch_pair(p1, p2).to(device)

with torch.no_grad():
    logits = model(input_tensor)
    pred = torch.argmax(logits, dim=1).item()

st.subheader("Guess the direction of the red patch (from blue):")
col1, col2, col3 = st.columns(3)
guessed = st.session_state.get("guessed", False)

def guess(direction_idx):
    st.session_state.clicked = direction_idx
    st.session_state.guessed = True

with col1:
    if st.button("Top-Left"): guess(3)
    if st.button("Left"): guess(4)
    if st.button("Bottom-Left"): guess(5)

with col2:
    if st.button("Top"): guess(2)
    st.write(" ")
    if st.button("Bottom"): guess(6)

with col3:
    if st.button("Top-Right"): guess(1)
    if st.button("Right"): guess(0)
    if st.button("Bottom-Right"): guess(7)

if st.session_state.get("guessed", False):
    user_dir = direction_names[st.session_state.clicked]
    correct_dir = direction_names[label]
    model_dir = direction_names[pred]
    st.markdown("---")
    if st.session_state.clicked == label:
        st.success(f"✅ Correct! You picked: **{user_dir}**")
    else:
        st.error(f"❌ Wrong. You picked: **{user_dir}** — Correct: **{correct_dir}**")
    st.info(f"🤖 Model predicted: **{model_dir}**")
    if st.button("Play Again"):
        st.session_state.guessed = False
        st.rerun()
